In [61]:
import pandas as pd
import re

In [62]:
emails = pd.read_csv('../data/mail_tbl -- most recent.csv')

In [63]:
#emails = pd.read_csv('../data/mail_tbl.csv')
emails.Body = emails.Body.fillna("") 

In [64]:
print(len(emails))
#emails.head()

2007


## Filtering Emails

---

1. Need something to keep track of thread ID (keep list of all thread IDs?)
2. When new email comes in, check whether ID belongs in list of ID's
3. If it does, check to see if STAFF or student initiated
4. If not, check to see if STAFF or student initiated, and if student, add ID to list of ID's
5. If Staff, send to "Important" Folder
6. Don't consider auto-generated emails ("has accepted invitation", from Google, etc)

---


We will only look at "Important Inbox"/"Inbox" label types

In [65]:
emails['Date'] = pd.to_datetime(emails['Date'])

In [66]:
#making list of all threat id's
thread_ids = emails['X-GM-THRID'].unique()

In [67]:
#list of all thread id's initiated by EE Staff
from_ee = []
for id in thread_ids:
    id_tbl = emails[emails['X-GM-THRID'] == id].sort_values('Date', ascending=True)
    if list(id_tbl['From'])[0] == 'STAFF <.>':
        from_ee.append(id)
print("Number of threads initiated by staff:", len(from_ee))

Number of threads initiated by staff: 89


In [68]:
#filter out any emails threads started by EE Staff
emails = emails[~emails['X-GM-THRID'].isin(from_ee)]

In [69]:
#all folders containing emails
emails['X-Gmail-Labels'].value_counts(dropna=False)

Important,Inbox                                  757
Important,Sent                                   315
Sent                                             288
Inbox                                             23
Unread,Important,Inbox                            22
Important,discuss at admin,Sent                   17
Important,discuss at admin,Inbox                  15
Important,Starred,Inbox                           15
Unread,Important,discuss at admin,Inbox           12
Unread,Important,discuss at admin,Sent            12
Important,Inbox,for                               10
Important,cheating,Inbox                           6
Unread,Important,Sent                              5
Important                                          4
Unread,Important,tabled,Inbox                      4
Important,Sent,for                                 4
discuss at admin,Sent                              3
Important,discuss at admin,Inbox,for               3
Important,for                                 

In [70]:
#not include spams
non_important_mails = ["Unread,Spam", "Drafts"]
emails = emails[(emails['X-Gmail-Labels'] != "Unread,Spam") & (emails['X-Gmail-Labels'] != "Drafts")]

In [71]:
#emails[emails['From'].str.contains('Google') == False]

In [72]:
#filtering out google emails and EE16A emails 
emails = emails[emails['From'].str.contains('Google')==False]

In [73]:
#filtering out emails where people accept invitations
emails = emails[~emails['Body'].str.contains('has accepted this invitation')]

In [74]:
print(len(emails))

1520


In [75]:
print(len(emails['X-GM-THRID'].unique()))
#emails.head()

403


# Fatou Anoymizing

In [48]:
def remove_mail_addresses(body):
    regex_mail = r"([a-z0-9.\-]+@(eecs\.)?berkeley\.edu)"
    pattern_mail = re.compile(regex_mail)
    try :
        body = pattern_mail.sub("", body)
    except:
        pass
    return body

In [49]:
#example
body = "hi grace, i am interested in being considered - i'll present on pagerank. thanks, oski bear. oski.bear@berkeley.edu "
print("Before:", body)
print("After:", remove_mail_addresses(body))

Before: hi grace, i am interested in being considered - i'll present on pagerank. thanks, oski bear. oski.bear@berkeley.edu 
After: hi grace, i am interested in being considered - i'll present on pagerank. thanks, oski bear.  


In [50]:
#remove emails
emails.Body = emails.Body.apply(remove_mail_addresses)
emails.From = emails.From.apply(remove_mail_addresses)
emails.To = emails.To.apply(remove_mail_addresses)
emails.Subject = emails.Subject.apply(remove_mail_addresses)
#emails.head()

In [51]:
import re

In [52]:
def remove_names(body):
    regex_end_expression = r"(sincerely|best|kind regards),?"
    pattern_end_expression = re.compile(regex_end_expression)
    try :
        search = pattern_end_expression.search(body)
        if not (search is None) :
            num_words = search.span()[0]
            body = body[:num_words]
        regex_start_email = r"(hi|dear|hello)[,]?[ ]? (.+?)(?=,)"
        body = re.sub(regex_start_email, r"\1", body)
        regex_class = r"(EE16A|EE16B)"
        body = re.sub(regex_class, "", body)
        body = re.sub(' +',' ',body)#remove extra spaces
    except :
        pass
    return body

In [53]:
#example
body = emails.Body.iloc[6]
print("Before:", body)
print("")
print("After:", remove_names(body))

Before: i meant we can see who didn't get in after the deadline has passed, so we wouldn't be able to add them in the spring.  i want to reiterate that i need to make changes to the schedule before the spring semester and not once it begins. i am on a limited schedule and cannot expand as late as we have done in previous semesters.  lydia  on thu, dec 14, 2017 at 1:51 pm, elad alon <> wrote:  > i certainly don’t want to create additional work either, but this semester > the drop rate for  was pretty low, so i’m not sure we should rely on > that to get people who need the class in.  so, can i suggest that we wait > to hear from lydia about the profile of people who are currently on the > waiting list? > thanks, > > elad > > on dec 14, 2017, at 1:34 pm, lydia raya <> wrote: > > hi , , > > i am sure that you and the  team have been working very hard already > for the spring and i don't want to create any more work for you. you are > right that students will drop and hopefully there won't 

In [54]:
#counter example 
tmp = emails.Body.iloc[3]

In [55]:
#remove names that appear at the end and at the beginning 
emails.Body = emails.Body.apply(remove_names)
emails.From = emails.From.apply(remove_names)
emails.To = emails.To.apply(remove_names)
emails.Subject = emails.Subject.apply(remove_names)
#emails.head()

In [76]:
emails.to_csv('../data/mail_tbl_anonymized_v3.csv', index=False, sep = ',', encoding='utf-8')

# Removing reply threads

In [90]:
e = pd.read_csv('../data/mail_tbl_anonymized_v3.csv')

In [91]:
e = e.fillna('')

In [92]:
len(e)

1520

In [93]:
import re

In [94]:
def remove_xtra(content):
    return re.sub("on ... [\d]{1,2}, [\d]{4}, at [\d]{1,2}:[\d]{1,2} .* | on ..., ... [\d]{1,2}, [\d]{4} at [\d]{1,2}:[\d]{1,2} .*", '', content)

In [95]:
e.Body[5]

'i certainly don’t want to create additional work either, but this semester the drop rate for  was pretty low, so i’m not sure we should rely on that to get people who need the class in.  so, can i suggest that we wait to hear from lydia about the profile of people who are currently on the waiting list? thanks,  elad  > on dec 14, 2017, at 1:34 pm, lydia raya <> wrote: >  > hi , , >  > i am sure that you and the  team have been working very hard already for the spring and i don\'t want to create any more work for you. you are right that students will drop and hopefully there won\'t end up being too many students who can\'t get into the class; those students will have summer as an option. i can take a look at the students who don\'t end up getting in to see who they are and if they need to be prioritized for future semesters. >  > michel &  staff, i will work on expanding  now.  >  > happy holidays to all, > lydia >  >> on thu, dec 14, 2017 at 1:08 pm,  <> wrote: >> hi lydia, >>  >> -  

In [96]:
e.Body = e.Body.apply(remove_xtra)

In [97]:
remove_xtra(e['Body'][5])

'i certainly don’t want to create additional work either, but this semester the drop rate for  was pretty low, so i’m not sure we should rely on that to get people who need the class in.  so, can i suggest that we wait to hear from lydia about the profile of people who are currently on the waiting list? thanks,  elad  > '

# Splitting DF into 5 for Labeling

In [99]:
e = e.sort_values(['X-GM-THRID', 'Date'])

In [101]:
length = len(e)//5
length

304

In [102]:
length = len(emails)//5

rohan = e.iloc[0:length, :]
keiko = e.iloc[length:2*length, :]
fatou = e.iloc[2*length:3*length, :]
kristian = e.iloc[3*length:4*length, :]
joyce = e.iloc[4*length:5*length, :]


In [103]:
rohan.to_csv('../data/rohan_label.csv', index=False, sep = ',', encoding='utf-8')
keiko.to_csv('../data/keiko_label.csv', index=False, sep = ',', encoding='utf-8')
fatou.to_csv('../data/fatou_label.csv', index=False, sep = ',', encoding='utf-8')
kristian.to_csv('../data/kristian_label.csv', index=False, sep = ',', encoding='utf-8')
joyce.to_csv('../data/joyce_label.csv', index=False, sep = ',', encoding='utf-8')

In [104]:
len(pd.read_csv('../data/rohan_label.csv'))

304